Imagine that you’re working on a new text-editor app. Your task is to create a toolbar with a bunch of buttons. 

You created a `Button` class that can be used for generic purpose.

![](../assets/command-problem1.png)

Each buttons is supposed to do different things.

The simplest solution is to create tons of subclasses for each button.

![](../assets/command-problem2.png)

This approach is deeply flawed. 

1. [ISSUE 1] You have an enormous number of subclasses, and you were risking breaking the code in these subclasses each time you modify the base `Button` class.
2. [ISSUE 2] Some operations might need to be invoked from multiple sources. For example, to copy-paste text, a user could click a "Copy" button, or via the context menu, or hitting `Ctrl+C`. You will have to duplicate the same code in multiple places.

A common implementation is like what you might usually do in MVC:

![](../assets/command-solution1-en.png)

Your GUI object ***calls a method*** of a Controller object. This process is usually described as one object ***sending a request*** to another object:

In [30]:
from abc import ABC, abstractmethod


class BaseButton(ABC):  # Base class for all buttons

    @abstractmethod
    def on_click(self):
        pass


class BaseKeyboardShorcut(ABC):  # Base class for all keyboard shortcuts
    
    @abstractmethod
    def on_press(self):
        pass


class ICopy(ABC):   # Interface (Not base class) for action copy

    @abstractmethod
    def copy(self, selected_text):   # Interface means code is not re-usable
        pass


class ButtonCopy(BaseButton, ICopy):  # inherit from BaseButton and implement ICopy

    def __init__(self, controller, view):
        self.controller = controller
        self.view = view

    def on_click(self):
        print('ButtonCopy: clicked')
        self.copy(self.view.display_text[self.view.selected_from:self.view.selected_to])

    def copy(self, selected_text):
        self.controller.save_to_clipboard(selected_text) # directly send a request


class CtrlC(BaseKeyboardShorcut, ICopy):  # inherit from BaseKeyboardShorcut and implement ICopy

    def __init__(self, controller, view):
        self.controller = controller
        self.view = view

    def on_press(self):
        print('CtrlC: pressed')
        self.copy(self.view.display_text[self.view.selected_from:self.view.selected_to])

    def copy(self, selected_text):
        self.controller.save_to_clipboard(selected_text) # directly send a request


class CTextEditor:  # Controller

    clipboard = None

    def save_to_clipboard(self, selected_text):
        print('CTextEditor: saved text to clipboard:', selected_text)
        self.clipboard = selected_text

    def get_from_clipboard(self):
        return self.clipboard


class VTextEditor:  # View

    display_text = "Hello LapTQ"
    selected_from = None
    selected_to = None  

    def __init__(
            self,
            controller
    ):
        # create buttons and shortcuts
        self.button_copy = ButtonCopy(controller, self)
        self.ctrl_c = CtrlC(controller, self)

        while True:
            action = input("""What do you want to do: 
                1. Select text
                2. Click Copy Button
                3. Press Ctrl + C
                q. Quit
                """
            )
            if action == "1":
                self.select_text()
            elif action == "2":
                self.click_copy_button()
            elif action == "3":
                self.press_ctrl_c()
            elif action == "q":
                break

    def select_text(self):
        self.selected_from = int(input("From: "))
        self.selected_to = int(input("To: "))

    def click_copy_button(self):
        self.button_copy.on_click()

    def press_ctrl_c(self):
        self.ctrl_c.on_press()
        

if __name__ == '__main__':

    controller = CTextEditor()
    
    VTextEditor(controller)

CtrlC: pressed
CTextEditor: saved text to clipboard: lo La


However, the Command pattern don't think that's a good idea. It says that the GUI object should ***not*** send the request ***directly***.

Instead, you should extract all the details of the request, such as:
* the object to send the request to,
* the name of the method to call,
* the arguments to pass along

into a separate *command* class. This *command* class will be responsible for triggering this request:

![](../assets/command-solution2-en.png)

In [25]:
from abc import ABC, abstractmethod


class BsseCommand(ABC):

    def __init__(self, controller, view):
        self.controller = controller
        self.view = view

    @abstractmethod
    def execute(self):
        pass


class CommandCopy(BsseCommand):

    def execute(self):    # deligate the copy action to this method
        print('CommandCopy: execute')
        self.controller.save_to_clipboard(self.view.display_text[self.view.selected_from:self.view.selected_to])


class Button:   # now, don't need subclass for each button

    def __init__(self, command):
        self.command = command

    def on_click(self):
        print('Button: clicked')
        self.command.execute()


class KeyboardShorcut:  # now, don't need subclass for each keyboard shortcut

    def __init__(self, command):
        self.command = command

    def on_press(self):
        print('KeyboardShortcut: pressed')
        self.command.execute()


class VTextEditor:

    display_text = "Hello LapTQ"
    selected_from = None
    selected_to = None

    def __init__(
            self,
            controller
    ):
        command_copy = CommandCopy(controller, self)
        
        # set the same command for both button and shortcut
        self.button_copy = Button(command_copy)
        self.ctrl_c = KeyboardShorcut(command_copy)

        while True:
            action = input("""What do you want to do: 
                1. Select text
                2. Click Copy Button
                3. Press Ctrl + C
                q. Quit
                """
            )
            if action == "1":
                self.select_text()
            elif action == "2":
                self.click_copy_button()
            elif action == "3":
                self.press_ctrl_c()
            elif action == "q":
                break

    def select_text(self):
        self.selected_from = int(input("From: "))
        self.selected_to = int(input("To: "))

    def click_copy_button(self):
        self.button_copy.on_click()

    def press_ctrl_c(self):
        self.ctrl_c.on_press()
        

if __name__ == '__main__':

    controller = CTextEditor()
    
    VTextEditor(controller)

Button: clicked
CommandCopy: execute
CTextEditor: saved text to clipboard: lo La


1. $\implies$ solve [ISSUE 1]: We don't have to subclass the `Button`.
2. $\implies$ solve [ISSUE 2]: We see that the same `CommandCopy` object can be used by multiple GUI objects (button, keyboard shortcut) without duplicating the same logic (the implementation of the `copy` method).

> [!TIP]
> When?
> 
> Avoid code duplication:
> * Between child classes of a common parent class by having no inheritance at all (no need to subclass `Button` for each button). Also avoid risk of breaking the code in these subclasses each time you modify the base `Button` class.
> * Between classes of different types (`ButtonCopy`, `CtrlC`) but sharing the same job (`copy`ing).

> [!TIP]
> How?
> By abstracting the job (`copy`ing) into a separate object (`CommandCopy`) $\implies$ decoupling the job from worker from the viewpoint of the View.

Using the Command pattern enables some interesting features:
1. Easily store command history.
2. Undo/Redo operations.

In [27]:
class BaseCommand(ABC):

    backup = []

    def __init__(self, controller, view):
        self.controller = controller
        self.view = view    # associate with view

    @abstractmethod
    def execute(self):
        pass

    @abstractmethod
    def undo(self):
        pass


class CommandCut(BaseCommand):    # take cuting as an example

    def execute(self):

        print('CommandCut: Text before cut:', self.view.display_text)
        self.backup.append(self.view.display_text)
        
        self.view.command_history.append(self)  # save the command to history
        
        # for simplicity, we use a naive way to cut text
        display_text = self.view.display_text[:self.view.selected_from] + self.view.display_text[self.view.selected_to:]
        self.view.display_text = display_text
        print('CommandCut: Text after cut:', display_text)

    def undo(self):
        display_text = self.backup.pop()
        self.view.display_text = display_text
        print('CommandCut: Text after undo:', display_text)
        

class VTextEditor:

    display_text = "Hello LapTQ"
    selected_from = None
    selected_to = None
    command_history = []


    def __init__(
            self,
            controller
    ):
        command_cut = CommandCut(controller, self)
        
        # set the same command for both button and shortcut
        self.button_cut = Button(command_cut)
        self.ctrl_c = KeyboardShorcut(command_cut)

        while True:
            action = input("""What do you want to do: 
                1. Select text
                2. Click Cut Button
                3. Undo
                q. Quit
                """
            )
            if action == "1":
                self.select_text()
            elif action == "2":
                self.click_copy_button()
            elif action == "3":
                self.undo()
            elif action == "q":
                break

    def select_text(self):
        self.selected_from = int(input("From: "))
        self.selected_to = int(input("To: "))

    def click_copy_button(self):
        self.button_cut.on_click()

    def undo(self):
        if len(self.command_history) == 0:
            return
        
        last_command = self.command_history.pop()
        last_command.undo()


if __name__ == '__main__':

    controller = CTextEditor()
    
    VTextEditor(controller)

Button: clicked
CommandCut: Text before cut: Hello LapTQ
CommandCut: Text after cut: HelpTQ
Button: clicked
CommandCut: Text before cut: HelpTQ
CommandCut: Text after cut: Hel
CommandCut: Text after undo: HelpTQ
CommandCut: Text after undo: Hello LapTQ


> [!TIP]
> When?
>
> Implement undo/redo

Although there are many ways to implement undo/redo, the Command pattern is perhaps ***the most popular*** of all.

This method has two drawbacks. First, it isn’t that easy to save an application’s state because some of it can be private. This problem can be mitigated with the Memento pattern.

Second, the state backups may consume quite a lot of RAM. Therefore, sometimes you can resort to an alternative implementation: instead of restoring the past state, the command performs the inverse operation. The reverse operation also has a price: it may turn out to be hard or even impossible to implement.

> [!TIP]
> When?
>
> Queue operations, schedule their execution, or execute them remotely.

As with any other object, a command can be serialized, which means converting it to a string that can be easily written to a file or a database. Later, the string can be restored as the initial command object.

## References

1. https://refactoring.guru/design-patterns/command